# Riqueza de solos

## Importando pacotes e inicializando *geemap*

In [1]:
import os
import ee
import geemap

In [2]:
geemap.ee_initialize()

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_doiqkQG3NJ1t8IS?source=API


A riqueza de solos é calculada como a quantidade de componentes da unidade de mapeamento de solos.

## Banco de Dados

Utilizamos a [base de dados de pedologia](https://geoftp.ibge.gov.br/informacoes_ambientais/pedologia/vetores/escala_250_mil/versao_2023/pedo_area.zip) do Instituto Brasileiro de Geografia e Estatística (IBGE)  para o cálculo da riqueza de solos. Calculamos a quantidade de solos componentes de cada polígono e rasterizados na escala do modelo digital de elevação utilizado nas etapas anteriores (~ 92 metros).

## Códigos para a criação da riqueza de solos

As análises foram rodadas no *Google Earth Engine* (Gorelick 2017), devido a demanda computacional do projeto, usando o pacote **geemap** (Wu 2020), em *Python* (Python Software Foundation 2023), como interface pela facilidade na documentação e reprodutividade das análises. O JupyterNotebook para a reprodução das análises pode ser encontrado em [https://github.com/Resiliencia-climatica-Brasil/diversity-resilience-python/blob/master/jupyternotebook/4_soil_diversity.ipynb](https://github.com/Resiliencia-climatica-Brasil/diversity-resilience-python/blob/master/jupyternotebook/4_soil_diversity.ipynb). 

O primeiro passo é criar uma função que conta os tipos de solos componentes de cada polígono. Concatenamos as colunas **componente**, **component1**, **component2** e **component3**. Depois, separamos a *string* pelos sinais de "+", que indicam as separações das descrições de cada tipo de solo. Em seguida, contamos quantos tipo de solos existem em cada polígono como o comprimento da lista de tipos de solos. Adicionamos uma coluna chamada **soil_richness** com a riqueza de solos em cada polígono.

In [3]:
def fun(feature):
    number = (ee.String(feature.get("componente"))
              .cat(ee.String(feature.get("component1")))
              .cat(ee.String(feature.get("component2")))
              .cat(ee.String(feature.get("component3")))
              .split("\+")
              .length())

    return feature.set("soil_richness", number)



Em seguida, importamos os dados de solo, definimos a região para exportar o dado e a escala da análise. 

In [4]:
# Importando os polígonos de solo
solo = ee.FeatureCollection("projects/ee-lucasljardim9/assets/pedo_area")

# Importando mapa de biomas do IBGE para extrair as coordenadas mínimas e máximas do Brasil
regiao = ee.FeatureCollection("projects/ee-lucasljardim9/assets/Biome")

def func_cmp(feature):
    return feature.bounds() 

# Extraindo as coordenadas mínimas e máximas do Brasil
regiao_box = regiao.map(func_cmp).geometry().dissolve(**{'maxError': 1}).bounds()

# Extraindo a resolução do mapa
DEM = ee.Image("MERIT/DEM/v1_0_3") 

escala = DEM.projection().nominalScale()

Aplicamos a função criada anteriormente aos polígonos de solo.

In [19]:
solo = solo.map(fun)

Rasterizamos a riqueza de solos na mesma escala das análises anteriores. 

In [20]:

soil_diversity = solo.reduceToImage(**{'properties':["soil_richness"],
                     'reducer':ee.Reducer.max()}) \
                     .reproject(**{'crs':"EPSG:4326",
                                  'scale':escala})

Por fim, exportamos o *raster* como um *asset* no *Google Earth Engine*.

In [ ]:
# Exporte o raster de riqueza de solo 
assetId = "projects/ee-lucasljardim9/assets/soil_diversity"

geemap.ee_export_image_to_asset(
    soil_diversity, description='soil_diversity', assetId=assetId, region=regiao_box, scale=escala, maxPixels=1e13
)